<a href="https://colab.research.google.com/github/03axdov/Notebooks/blob/main/HeartDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

In [2]:
csv_file = tf.keras.utils.get_file("heart.csv", "https://storage.googleapis.com/download.tensorflow.org/data/heart.csv")

24576/13273 [=======================================================] - 0s 0us/step


In [3]:
df = pd.read_csv(csv_file)

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [5]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [6]:
target = df.pop("target")

In [7]:
numeric_features_names = ["age", "thalach", "trestbps", "chol", "oldpeak"]
numeric_features = df[numeric_features_names]
numeric_features.head()

,age,thalach,trestbps,chol,oldpeak
0,63,150,145,233,2.3
1,67,108,160,286,1.5
2,67,129,120,229,2.6
3,37,187,130,250,3.5
4,41,172,130,204,1.4


In [8]:
tf.convert_to_tensor(numeric_features)

<tf.Tensor: shape=(303, 5), dtype=float64, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]])>

In [9]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [10]:
print(numeric_features.iloc[:3])

   age  thalach  trestbps  chol  oldpeak
0   63      150       145   233      2.3
1   67      108       160   286      1.5
2   67      129       120   229      2.6


In [11]:
print(numeric_features[:3])

   age  thalach  trestbps  chol  oldpeak
0   63      150       145   233      2.3
1   67      108       160   286      1.5
2   67      129       120   229      2.6


In [12]:
normalizer(numeric_features.iloc[:3])

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.93383914,  0.03480718,  0.74578077, -0.2600867 ,  1.0680453 ],
       [ 1.3782105 , -1.7806165 ,  1.5923285 ,  0.7573879 ,  0.38022864],
       [ 1.3782105 , -0.87290466, -0.6651321 , -0.3368772 ,  1.3259765 ]],
      dtype=float32)>

In [13]:
def get_basic_model():
  model = tf.keras.Sequential([
      normalizer,
      tf.keras.layers.Dense(10, activation="elu"),
      tf.keras.layers.Dense(10, activation="elu"),
      tf.keras.layers.Dense(1)
  ])

  model.compile(
      optimizer = tf.keras.optimizers.Adam(),
      loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
      metrics = ["accuracy"]
  )

  return model

In [14]:
model = get_basic_model()
model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
152/152 [==============================] - 1s 1ms/step - loss: 0.5905 - accuracy: 0.7393
Epoch 2/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4954 - accuracy: 0.7789
Epoch 3/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4674 - accuracy: 0.7855
Epoch 4/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4558 - accuracy: 0.7690
Epoch 5/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4501 - accuracy: 0.7690
Epoch 6/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4471 - accuracy: 0.7723
Epoch 7/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4460 - accuracy: 0.7723
Epoch 8/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4445 - accuracy: 0.7723
Epoch 9/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4442 - accuracy: 0.7657
Epoch 10/15
152/152 [==============================] - 0s 1ms/step - loss: 0.4436 - accuracy: 0.7690

In [15]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(5):
  print(row)

(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 63. , 150. , 145. , 233. ,   2.3])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 108. , 160. , 286. ,   1.5])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 129. , 120. , 229. ,   2.6])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 37. , 187. , 130. , 250. ,   3.5])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 41. , 172. , 130. , 204. ,   1.4])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [16]:
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()

import time
tic = time.time()
model.fit(numeric_batches, epochs=15)
toc = time.time()
print(f"Time: {toc - tic}")

Epoch 1/15
152/152 [==============================] - 1s 2ms/step - loss: 0.7310 - accuracy: 0.6601
Epoch 2/15
152/152 [==============================] - 0s 2ms/step - loss: 0.5646 - accuracy: 0.7129
Epoch 3/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4985 - accuracy: 0.7261
Epoch 4/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4689 - accuracy: 0.7525
Epoch 5/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4542 - accuracy: 0.7657
Epoch 6/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4505 - accuracy: 0.7888
Epoch 7/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4455 - accuracy: 0.7822
Epoch 8/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4429 - accuracy: 0.7987
Epoch 9/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4432 - accuracy: 0.7888
Epoch 10/15
152/152 [==============================] - 0s 2ms/step - loss: 0.4421 - accuracy: 0.7888

In [17]:
numeric_dict_ds = tf.data.Dataset.from_tensor_slices((dict(numeric_features), target))

for row in numeric_dict_ds.take(3):
  print(row)

({'age': <tf.Tensor: shape=(), dtype=int64, numpy=63>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=150>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=145>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=233>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=2.3>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
({'age': <tf.Tensor: shape=(), dtype=int64, numpy=67>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=108>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=160>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=286>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=1.5>}, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
({'age': <tf.Tensor: shape=(), dtype=int64, numpy=67>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=129>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=120>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=229>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=2.6>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [18]:
def stack_dict(inputs, fun=tf.stack):
  values = []
  for key in sorted(inputs.keys()):
    values.append(tf.cast(inputs[key], tf.float32))

  return fun(values, axis=-1)

In [19]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__(self)

    self.normalizer = tf.keras.layers.Normalization(axis=-1)

    self.seq = tf.keras.Sequential([
        self.normalizer,
        tf.keras.layers.Dense(10, activation="elu"),
        tf.keras.layers.Dense(10, activation="elu"),
        tf.keras.layers.Dense(1)
    ])

  def adapt(self, inputs):
    inputs = stack_dict(inputs)
    self.normalizer.adapt(inputs)

  def call(self, inputs):
    inputs = stack_dict(inputs)
    result = self.seq(inputs)

    return result

  
model = MyModel()

model.adapt(dict(numeric_features))
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = ["accuracy"],
    run_eagerly=True
)

In [20]:
model.fit(
    dict(numeric_features), target, epochs=5, batch_size=BATCH_SIZE
)

Epoch 1/5
152/152 [==============================] - 2s 13ms/step - loss: 0.5716 - accuracy: 0.7393
Epoch 2/5
152/152 [==============================] - 2s 12ms/step - loss: 0.4813 - accuracy: 0.7723
Epoch 3/5
152/152 [==============================] - 2s 12ms/step - loss: 0.4575 - accuracy: 0.7690
Epoch 4/5
152/152 [==============================] - 2s 12ms/step - loss: 0.4493 - accuracy: 0.7789
Epoch 5/5
152/152 [==============================] - 2s 12ms/step - loss: 0.4450 - accuracy: 0.7756


In [21]:
numeric_dict_batches = numeric_dict_ds.shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE)
model.fit(numeric_dict_batches, epochs=5)

Epoch 1/5
152/152 [==============================] - 1s 10ms/step - loss: 0.4427 - accuracy: 0.7690
Epoch 2/5
152/152 [==============================] - 1s 10ms/step - loss: 0.4414 - accuracy: 0.7723
Epoch 3/5
152/152 [==============================] - 2s 10ms/step - loss: 0.4404 - accuracy: 0.7756
Epoch 4/5
152/152 [==============================] - 2s 10ms/step - loss: 0.4410 - accuracy: 0.7756
Epoch 5/5
152/152 [==============================] - 2s 11ms/step - loss: 0.4406 - accuracy: 0.7756


In [23]:
model.predict(dict(numeric_features.iloc[:3]))

array([[[0.15212858]],

       [[0.871331  ]],

       [[0.3800615 ]]], dtype=float32)

In [24]:
"""inputs = {}
for name,column in numeric_features.items():
  inputs[name] = tf.keras.Input(
      shape=(1,), name=name, dtype=tf.float32
  )"""

'inputs = {}\nfor name,column in numeric_features.items():\n  inputs[name] = tf.keras.Input(\n      shape=(1,), name=name, dtype=tf.float32\n  )'